In [1]:
from PrjtCF_model_pkg.config import *
import importlib

In [3]:
from PrjtCF_prjt_data import InputData_1 as D2

In [27]:
importlib.reload(D2)

<module 'FS_dangjin_data.InputData_2' from '/Users/KP_Hong/Documents/PyPrjt/[lib]DangjinLogistic/FS_dangjin_data/InputData_2.py'>

# Data Summary

In [4]:
print("Total Sales Amount : {:,.0f}".format(D2.sales.amt().sum()))
print("Total Cost Fixed Amount : {:,.0f}".format(D2.cstfxd.amt().sum()))
print("Total Cost for Sale Amount : {:,.0f}".format(D2.cstfsl.amt().sum()))
print("Total Financial Cost Amount : {:,.0f}".format(D2.cstfnc.amt().sum()))

Total Sales Amount : 243,662
Total Cost Fixed Amount : 124,126
Total Cost for Sale Amount : 26,929
Total Financial Cost Amount : 2,587


# Cash Flow

In [8]:
importlib.reload(D2)

# read basic data
ttlsales = D2.sales.amt().sum()

# make empty cashflow dataframe
columns = ['BAL_INTL', 'sales_paid', 'sales_rate', 'TrA_paid', 'TrB_paid', 'INFLWD_AMT',
           'cstfnc_paid', 'TrA_feerepd', 'TrB_feerepd', 'TrA_IRrepd', 'TrB_IRrepd', 'OUTFLWD_FN',
           'cstfxd_paid', 'cstfsl_paid', 'OUTFLWD_CST', 
           'NET_OPRTN',
           'TrA_amtrepd', 'TrB_amtrepd', 'OUTFLWD_NTL',
           'BAL_CLSN']
index = np.arange(-1, D2.mtrt + 1)
cashflow = pd.DataFrame(np.zeros([len(index), len(columns)]), columns=columns, index=index)

# make accounts
acc_repmt = D2.acc.account(D2.mtrt)
acc_oprtn = D2.acc.account(D2.mtrt)

# start writing cash flow
for idx in np.arange(0, D2.mtrt+1):
    cashflow.loc[idx, 'BAL_INTL'] = acc_oprtn.bal_intl(idx) + acc_repmt.bal_intl(idx)

    #### SALES AMT INFLOWED ####
    # pay sales amount
    for dct in D2.sales_dct:
        D2.sales_dct[dct].amt_pay(idx) # <- 분양대금 입금 일정 추가 필요 : 분양대금 입금 스케줄, 실 분양계약 발생일, 분양계약 발생에 따른 분양대금 입금 스케줄
    cashflow.loc[idx, 'sales_paid'] = D2.sales.df('amt_paid').loc[idx].sum()
    cashflow.loc[idx, 'sales_rate'] = cashflow.loc[idx, 'sales_paid'] / ttlsales

    # sum up the results
    acc_repmt.addamt(idx, cashflow.loc[idx, 'sales_paid'] * D2.repybal_rate)
    acc_oprtn.addamt(idx, cashflow.loc[idx, 'sales_paid'] * (1 - D2.repybal_rate))
    cashflow.loc[idx, 'INFLWD_AMT'] = cashflow.loc[idx, 'sales_paid']
    #### SALES AMT INFLOWED ####


    #### TRANSFER AMOUNT ON REPAYMENT ACCOUNT TO OPERATING ACCOUNT ####
    amt_trsfr_TrA = min(acc_repmt.bal_clsn(idx), D2.TrA.amt_wtdrwbl(idx))
    acc_repmt.subamt(idx, amt_trsfr_TrA)
    acc_oprtn.addamt(idx, amt_trsfr_TrA)
    D2.TrA.amt_rdc(idx, amt_trsfr_TrA)
    
    if D2.TrA.is_repaid() is True:
        amt_trsfr_TrB = min(acc_repmt.bal_clsn(idx), D2.TrB.amt_wtdrwbl(idx))
        acc_repmt.subamt(idx, amt_trsfr_TrB)
        acc_oprtn.addamt(idx, amt_trsfr_TrB)
        D2.TrB.amt_rdc(idx, amt_trsfr_TrB)
    #### TRANSFER AMOUNT ON REPAYMENT ACCOUNT TO OPERATING ACCOUNT ####
    
    
    #### ESTIMATE FINANCIAL COST AMOUNT ####
    # financial fee amt
    feeamt_TrA = D2.TrA.df.loc[idx, 'fee_unpd']
    feeamt_TrB = D2.TrB.df.loc[idx, 'fee_unpd']
    
    # IR to pay amt
    cstamt_fnc = sum([D2.cstfnc_dct[key].cost_amt(idx) for key in D2.cstfnc_dct])
    IRamt_TrA = D2.TrA.IR_rsdl(idx)
    IRamt_TrB = D2.TrB.IR_rsdl(idx)
    
    # sum up
    ttlamt_fnc = feeamt_TrA + feeamt_TrB + cstamt_fnc + IRamt_TrA + IRamt_TrB
    #### ESTIMATE FINANCIAL COST AMOUNT ####
    
    
    #### ESTIMATE FIXED/FOR-SALE COST AMOUNT ####
    cstamt_fxd = sum([D2.cstfxd_dct[key].cost_amt(idx) for key in D2.cstfxd_dct])
    cstamt_fsl = sum([D2.cstfsl_dct[key].amt * cashflow.loc[idx, 'sales_rate'] for key in D2.cstfsl_dct])
    #### ESTIMATE FIXED/FOR-SALE COST AMOUNT ####
    
    
    #### ESTIMATE SHORT TO PAY AND PAY LOAN ####
    ttlamt_topay = ttlamt_fnc + cstamt_fxd + cstamt_fsl
    short_amt = ttlamt_topay - acc_oprtn.bal_clsn(idx)
    
    tmpamt_rqrd = 0
    if short_amt > 0:
        payamt_min = 1_000
        tmpamt_rqrd = np.ceil(short_amt / payamt_min) * payamt_min

        # withdraw Tr.B amount
        tmpamt_topay = min(D2.TrB.amt_wtdrwbl(idx), tmpamt_rqrd)
        D2.TrB.amt_pay(idx, tmpamt_topay) # withdraw loan
        cashflow.loc[idx, 'TrB_paid'] = acc_oprtn.addamt(idx, D2.TrB.df.loc[idx, 'amt_paid'])
            # => add amount amt_paid to operating account and apply to cashflow dataframe
        tmpamt_rqrd = max(tmpamt_rqrd - tmpamt_topay, 0)

        # withdraw Tr.A amount
        tmpamt_topay = min(D2.TrA.amt_wtdrwbl(idx), tmpamt_rqrd)
        D2.TrA.amt_pay(idx, tmpamt_topay) # withdraw loan
        cashflow.loc[idx, 'TrA_paid'] = acc_oprtn.addamt(idx, D2.TrA.df.loc[idx, 'amt_paid'])
            # => add amount amt_paid to operating account and apply to cashflow dataframe
        tmpamt_rqrd = max(tmpamt_rqrd - tmpamt_topay, 0)
        
    cashflow.loc[idx, 'INFLWD_AMT'] = cashflow.loc[idx, 'INFLWD_AMT'] \
                                      + cashflow.loc[idx, 'TrB_paid'] + cashflow.loc[idx, 'TrA_paid']
    #### ESTIMATE SHORT TO PAY AND PAY LOAN ####

    
    #### PAY FINANCIAL COST AMOUNT ####
    tmpamt_cum = 0
    
    # repay financial cost amount
    for key in D2.cstfnc_dct:
        D2.cstfnc_dct[key].amt_pay(idx)
    tmpamt_torepy = D2.cstfnc.df('amt_paid').loc[idx].sum()
    tmpamt_cum += tmpamt_torepy
    acc_oprtn.subamt(idx, tmpamt_torepy) # withdraw repayment amt from operating account
    cashflow.loc[idx, 'cstfnc_paid'] = tmpamt_torepy # apply to cashflow dataframe

    # repay fee amt
    D2.TrA.fee_repay(idx, feeamt_TrA)
    tmpamt_torepy = D2.TrA.df.loc[idx, 'fee_repd']
    tmpamt_cum += tmpamt_torepy
    acc_oprtn.subamt(idx, tmpamt_torepy) # withdraw repayment amt from operating account
    cashflow.loc[idx, 'TrA_feerepd'] = tmpamt_torepy # apply to cashflow dataframe
    
    D2.TrB.fee_repay(idx, feeamt_TrB)
    tmpamt_torepy = D2.TrB.df.loc[idx, 'fee_repd']
    tmpamt_cum += tmpamt_torepy
    acc_oprtn.subamt(idx, tmpamt_torepy) # withdraw repayment amt from operating account
    cashflow.loc[idx, 'TrB_feerepd'] = tmpamt_torepy # apply to cashflow dataframe
    
    D2.TrA.IR_repay(idx, IRamt_TrA)
    tmpamt_torepy = D2.TrA.df.loc[idx, 'IR_repd']
    tmpamt_cum += tmpamt_torepy
    acc_oprtn.subamt(idx, tmpamt_torepy) # withdraw repayment amt from operating account
    cashflow.loc[idx, 'TrA_IRrepd'] = tmpamt_torepy # apply to cashflow dataframe

    D2.TrB.IR_repay(idx, IRamt_TrB)
    tmpamt_torepy = D2.TrB.df.loc[idx, 'IR_repd']
    tmpamt_cum += tmpamt_torepy
    acc_oprtn.subamt(idx, tmpamt_torepy) # withdraw repayment amt from operating account
    cashflow.loc[idx, 'TrB_IRrepd'] = tmpamt_torepy # apply to cashflow dataframe

    # sum up the results
    cashflow.loc[idx, 'OUTFLWD_FN'] = tmpamt_cum
    #### PAY FINANCIAL COST AMOUNT ####


    #### PAY FIXED AND FOR-SALE COST AMOUNT ####
    tmpamt_cum = 0

    for dct in D2.cstfxd_dct:
        D2.cstfxd_dct[dct].amt_pay(idx)
    tmpamt_topay = D2.cstfxd.df('amt_paid').loc[idx].sum()
    tmpamt_cum += tmpamt_topay
    acc_oprtn.subamt(idx, tmpamt_topay) # withdraw payment amt from operating account
    cashflow.loc[idx, 'cstfxd_paid'] = tmpamt_topay # apply to cashflow dataframe

    for dct in D2.cstfsl_dct:
        tmp_amt = D2.cstfsl_dct[dct].amt * cashflow.loc[idx, 'sales_rate']
        D2.cstfsl_dct[dct].amt_pay(idx, tmp_amt)
    tmpamt_topay = D2.cstfsl.df('amt_paid').loc[idx].sum()
    tmpamt_cum += tmpamt_topay
    acc_oprtn.subamt(idx, tmpamt_topay) # withdraw payment amt from operating account
    cashflow.loc[idx, 'cstfsl_paid'] = tmpamt_topay # apply to cashflow dataframe

    # sum up the results
    cashflow.loc[idx, 'OUTFLWD_CST'] = tmpamt_cum
    #### PAY FIXED AND FOR-SALE COST AMOUNT ####


    #### REPAY NOTIONAL AMOUNT ####
    amt_repmt_TrA = min(acc_repmt.bal_clsn(idx), D2.TrA.amt_rsdl(idx))
    acc_repmt.subamt(idx, amt_repmt_TrA)
    D2.TrA.amt_repay(idx, amt_repmt_TrA)
    cashflow.loc[idx, 'TrA_amtrepd'] = D2.TrA.df.loc[idx, 'amt_repd']
    
    if D2.TrA.is_repaid() is True:
        amt_repmt_TrB = min(acc_repmt.bal_clsn(idx), D2.TrB.amt_rsdl(idx))
        acc_repmt.subamt(idx, amt_repmt_TrB)
        D2.TrB.amt_repay(idx, amt_repmt_TrB)
        cashflow.loc[idx, 'TrB_amtrepd'] = D2.TrB.df.loc[idx, 'amt_repd']
        
    cashflow.loc[idx, 'OUTFLWD_NTL'] = cashflow.loc[idx, 'TrA_amtrepd'] + cashflow.loc[idx, 'TrB_amtrepd']
    #### REPAY NOTIONAL AMOUNT ####

    
    #### CLOSING ####
    cashflow.loc[idx, 'NET_OPRTN'] = cashflow.loc[idx, 'INFLWD_AMT'] - cashflow.loc[idx, 'OUTFLWD_FN'] - cashflow.loc[idx, 'OUTFLWD_CST']
    cashflow.loc[idx, 'BAL_CLSN'] = acc_oprtn.bal_clsn(idx) + acc_repmt.bal_clsn(idx)
    #### CLOSING ####
    
    
B(cashflow)

,BAL_INTL,sales_paid,sales_rate,TrA_paid,TrB_paid,INFLWD_AMT,cstfnc_paid,TrA_feerepd,TrB_feerepd,TrA_IRrepd,TrB_IRrepd,OUTFLWD_FN,cstfxd_paid,cstfsl_paid,OUTFLWD_CST,NET_OPRTN,TrA_amtrepd,TrB_amtrepd,OUTFLWD_NTL,BAL_CLSN
-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,"27,000","30,000","57,000","2,587","1,870","1,320",0,0,"5,777","50,266",0,"50,266",956,0,0,0,956
1,956,"24,366",0,0,0,"24,366",0,0,0,97,130,227,"1,835","2,693","4,528","19,611",0,0,0,"20,568"
2,"20,568",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"18,506"
3,"18,506",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"16,444"
4,"16,444",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"14,382"
5,"14,382",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"12,320"
6,"12,320",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"10,258"
7,"10,258",0,0,0,0,0,0,0,0,97,130,227,"1,835",0,"1,835","-2,062",0,0,0,"8,196"
8,"8,196",0,0,0,0,0,0,0,0,97,130,227,"2,285",0,"2,285","-2,512",0,0,0,"5,684"


In [9]:
D2.TrA.df

,amt_ntnl,amt_rdcd,amt_rdcd_cum,amt_paid,amt_paid_cum,amt_wtdrwbl,amt_repy_scd,amt_repd,amt_repd_cum,amt_rsdl,amt_paid_forIR,IR_rate,IR_amt,IR_amt_cum,IR_repd,IR_repd_cum,IR_rsdl,fee_amt,fee_repd,fee_unpd
-1,110000.0,0.000000,0.000000,0.0,0.0,110000.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.043,0.000000,0.000000,0.000000,0.000000,0.0,1870.0,0.0,1870.0
0,110000.0,0.000000,0.000000,27000.0,27000.0,83000.000000,0.0,0.000000,0.000000,27000.000000,0.000000,0.043,0.000000,0.000000,0.000000,0.000000,0.0,1870.0,1870.0,0.0
1,110000.0,19492.971503,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,96.750000,96.750000,96.750000,0.0,1870.0,0.0,0.0
2,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,193.500000,96.750000,193.500000,0.0,1870.0,0.0,0.0
3,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,290.250000,96.750000,290.250000,0.0,1870.0,0.0,0.0
4,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,387.000000,96.750000,387.000000,0.0,1870.0,0.0,0.0
5,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,483.750000,96.750000,483.750000,0.0,1870.0,0.0,0.0
6,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,580.500000,96.750000,580.500000,0.0,1870.0,0.0,0.0
7,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,677.250000,96.750000,677.250000,0.0,1870.0,0.0,0.0
8,110000.0,0.000000,19492.971503,0.0,27000.0,63507.028497,0.0,0.000000,0.000000,27000.000000,27000.000000,0.043,96.750000,774.000000,96.750000,774.000000,0.0,1870.0,0.0,0.0


In [102]:
acc_repmt.df

,bal_intl,amt_add,amt_sub,bal_clsn
-1,0.00000,0.000000,0.000000,0.00000
0,0.00000,0.000000,0.000000,0.00000
1,0.00000,19492.971503,19492.971503,0.00000
2,0.00000,0.000000,0.000000,0.00000
3,0.00000,0.000000,0.000000,0.00000
4,0.00000,0.000000,0.000000,0.00000
5,0.00000,0.000000,0.000000,0.00000
6,0.00000,0.000000,0.000000,0.00000
7,0.00000,0.000000,0.000000,0.00000
8,0.00000,0.000000,0.000000,0.00000
